In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, to_date, desc
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# Initialize Spark Session
spark = SparkSession.builder.appName("AdvancedEDA").getOrCreate()


In [ ]:
# Load datasets
fact_sales = spark.read.csv("FactInternetSales.csv", header=True, inferSchema=True)
dim_customer = spark.read.csv("DimCustomer.csv", header=True, inferSchema=True)
dim_product = spark.read.csv("DimProduct.csv", header=True, inferSchema=True)
dim_date = spark.read.csv("DimDate.csv", header=True, inferSchema=True)
dim_sales_territory = spark.read.csv("DimSalesTerritory.csv", header=True, inferSchema=True)


In [ ]:
# Cache datasets for performance
fact_sales.cache()
dim_customer.cache()
dim_product.cache()
dim_date.cache()
dim_sales_territory.cache()


In [ ]:
# Register DataFrames as SQL temporary views
fact_sales.createOrReplaceTempView("FactInternetSales")
dim_customer.createOrReplaceTempView("DimCustomer")
dim_product.createOrReplaceTempView("DimProduct")
dim_date.createOrReplaceTempView("DimDate")
dim_sales_territory.createOrReplaceTempView("DimSalesTerritory")


In [ ]:
# Step 1: Total Sales by Product
total_sales_by_product = spark.sql("""
    SELECT dp.EnglishProductName, SUM(fs.SalesAmount) AS TotalSales
    FROM FactInternetSales fs
    JOIN DimProduct dp ON fs.ProductKey = dp.ProductKey
    GROUP BY dp.EnglishProductName
    ORDER BY TotalSales DESC
    LIMIT 10
""")
total_sales_by_product.show()


In [ ]:
# Step 2: Monthly Sales Trend
monthly_sales_trend = spark.sql("""
    SELECT dd.CalendarYear, dd.EnglishMonthName, SUM(fs.SalesAmount) AS MonthlySales
    FROM FactInternetSales fs
    JOIN DimDate dd ON fs.OrderDateKey = dd.DateKey
    GROUP BY dd.CalendarYear, dd.EnglishMonthName
    ORDER BY dd.CalendarYear, dd.EnglishMonthName
""")
monthly_sales_trend.show()


In [ ]:
# Step 3: Top 5 Regions by Sales
top_regions = spark.sql("""
    SELECT dst.SalesTerritoryRegion, SUM(fs.SalesAmount) AS RegionalSales
    FROM FactInternetSales fs
    JOIN DimSalesTerritory dst ON fs.SalesTerritoryKey = dst.SalesTerritoryKey
    GROUP BY dst.SalesTerritoryRegion
    ORDER BY RegionalSales DESC
    LIMIT 5
""")
top_regions.show()

In [ ]:
# Step 4: Customer Segmentation by Income and Sales
customer_segmentation = spark.sql("""
    SELECT dc.YearlyIncome, COUNT(fs.CustomerKey) AS CustomerCount, SUM(fs.SalesAmount) AS TotalSales
    FROM FactInternetSales fs
    JOIN DimCustomer dc ON fs.CustomerKey = dc.CustomerKey
    GROUP BY dc.YearlyIncome
    ORDER BY TotalSales DESC
""")
customer_segmentation.show()


In [ ]:
# Step 5: Visualize SQL Query Results
# Convert to Pandas for plotting
df_top_regions = top_regions.toPandas()
df_total_sales_by_product = total_sales_by_product.toPandas()


In [ ]:
# Bar chart of top regions by sales
df_top_regions.plot(kind="bar", x="SalesTerritoryRegion", y="RegionalSales", color="purple", title="Top 5 Regions by Sales")
plt.show()

In [ ]:
# Bar chart of top products by sales
df_total_sales_by_product.plot(kind="bar", x="EnglishProductName", y="TotalSales", color="blue", title="Top 10 Products by Sales")
plt.show()

In [ ]:
# Scatter plot of Monthly Sales Trend
monthly_sales_trend_pandas = monthly_sales_trend.toPandas()
sns.lineplot(data=monthly_sales_trend_pandas, x="EnglishMonthName", y="MonthlySales", hue="CalendarYear")
plt.title("Monthly Sales Trend")
plt.show()

In [ ]:
# Clean up
spark.stop()
